In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
import graphviz
import dtreeviz
import pandas as pd
import numpy as np
import pickle
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree
from pypinyin import pinyin, lazy_pinyin, Style
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
# autogluon保存路径
model_path = r"F:\cache_data\zone_ana\dy\modle\autogluon_type_class"
# 检查路径是否存在，否则便创建
if not os.path.exists(model_path):
    os.makedirs(model_path)
else:
    print("文件夹已存在")

文件夹已存在


In [3]:
dataset = pd.read_csv(r"F:\cache_data\zone_ana\dy\train_data\result.csv")

In [4]:
dataset.columns

Index(['FID', 'DLMC', 'new_TZ', 'XMin', 'YMin', 'XMax', 'YMax', 'Centroid_X',
       'Centroid_Y', 'Riverdis',
       ...
       'MRVBF_COUNT', 'MRVBF_AREA', 'MRVBF_MIN', 'MRVBF_MAX', 'MRVBF_RANGE',
       'MRVBF_MEAN', 'MRVBF_STD', 'MRVBF_SUM', 'MRVBF_MEDIAN', 'MRVBF_PCT90'],
      dtype='object', length=401)

In [5]:
dataset["TZ_label"] = dataset.new_TZ.astype("category").cat.codes

In [6]:
dataset['DLMC'] = dataset['DLMC'].astype('category')
dataset['TZ_label'] = dataset['TZ_label'].astype('category')

In [7]:
result = dataset.groupby('TZ_label', observed=True)["new_TZ"].apply(lambda x: list(x.unique())).to_dict()
for one_type in result:
    result[one_type] = result[one_type][0]
print(result)

{0: '中层灰泥质黄色石灰土', 1: '中层砂泥质黄壤', 2: '厚层砂泥质黄壤', 3: '厚层红泥质黄壤', 4: '厚层黏质黄色石灰土', 5: '浅石灰泥田', 6: '浅石红泥田', 7: '潮泥田', 8: '潮砂泥田', 9: '石灰泥田', 10: '石红泥田', 11: '腐中层壤质黄色石灰土', 12: '腐中层壤质黑色石灰土', 13: '腐中层灰泥质黄壤', 14: '腐中层砂泥质黄壤', 15: '腐中层砂泥质黄壤性土', 16: '腐中层砾壤质酸性紫色土', 17: '腐中层砾壤质钙质紫色土', 18: '腐中层硅质黄壤', 19: '腐中层红泥质黄壤', 20: '腐厚层壤质黄色石灰土', 21: '腐厚层壤质黑色石灰土', 22: '腐厚层灰泥质黄壤', 23: '腐厚层砂泥质黄壤', 24: '腐厚层硅质黄壤', 25: '腐厚层红泥质黄壤', 26: '腐薄层壤质黄色石灰土', 27: '腐薄层灰泥质黄壤', 28: '腐薄层砂泥质黄壤', 29: '腐薄层砾砂质黑色石灰土', 30: '腐薄层硅质山地灌丛草甸土', 31: '腐薄层硅质黄壤', 32: '腐薄层红泥质黄壤', 33: '腐薄层黏质黄色石灰土', 34: '薄层壤质中性紫色土', 35: '薄层灰泥质黄色石灰土', 36: '薄层砂泥质黄壤', 37: '薄层砾壤质中性紫色土', 38: '薄层砾壤质黄色石灰土', 39: '薄层砾壤质黑色石灰土', 40: '薄层砾硅质黄壤', 41: '薄层硅质酸性粗骨土', 42: '轻漂白粉泥田', 43: '重漂砂泥田', 44: '重漂红泥田', 45: '青石灰泥田', 46: '黄浅白粉泥田', 47: '黄浅砂泥田', 48: '黄浅红泥田', 49: '黄浅鳝泥田', 50: '黄白粉泥田', 51: '黄砂泥田', 52: '黄红泥田', 53: '黄青白粉泥田', 54: '黄鳝泥田', 55: '黄黄砂泥田'}


In [8]:
# 保存映射关系
with open(r'D:\worker_code\Terrain_Test\data\soil_dict.json', 'w') as f:
    json.dump(result, f)

In [9]:
# 筛选水稻土数据
sdt_data = dataset[dataset['DLMC'].isin(['水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂'])]

In [10]:
pd.unique(sdt_data['new_TZ'])

array(['黄浅白粉泥田', '黄鳝泥田', '浅石红泥田', '石红泥田', '潮砂泥田', '石灰泥田', '黄红泥田', '青石灰泥田',
       '黄黄砂泥田', '轻漂白粉泥田', '黄青白粉泥田', '黄浅红泥田', '黄白粉泥田', '黄浅砂泥田', '重漂红泥田',
       '潮泥田', '浅石灰泥田', '黄砂泥田', '黄浅鳝泥田', '重漂砂泥田'], dtype=object)

In [11]:
# 筛选非自然土数据
fzrt_data = dataset[~dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地','水田','水浇地','坑塘水面','养殖坑塘','内陆滩涂'])]

In [12]:
pd.unique(fzrt_data['new_TZ']),fzrt_data.shape

(array(['薄层砾硅质黄壤', '厚层砂泥质黄壤', '中层灰泥质黄色石灰土', '厚层红泥质黄壤', '中层砂泥质黄壤',
        '薄层硅质酸性粗骨土', '薄层灰泥质黄色石灰土', '厚层黏质黄色石灰土', '薄层壤质中性紫色土', '薄层砾壤质黑色石灰土',
        '薄层砂泥质黄壤', '薄层砾壤质黄色石灰土', '薄层砾壤质中性紫色土'], dtype=object),
 (1075, 402))

In [13]:
# 筛选自然土数据
zrt_data = dataset[dataset['DLMC'].isin(['乔木林地','灌木林地','竹林地','其他林地','其他草地','天然牧草地','人工牧草地'])]

In [14]:
pd.unique(zrt_data['new_TZ']),zrt_data.shape

(array(['腐厚层砂泥质黄壤', '腐薄层灰泥质黄壤', '腐中层砂泥质黄壤', '腐中层壤质黄色石灰土', '腐薄层硅质黄壤',
        '腐薄层硅质山地灌丛草甸土', '腐中层硅质黄壤', '腐薄层砂泥质黄壤', '腐薄层壤质黄色石灰土', '腐薄层砾砂质黑色石灰土',
        '腐厚层硅质黄壤', '腐中层壤质黑色石灰土', '腐厚层红泥质黄壤', '腐薄层黏质黄色石灰土', '腐厚层灰泥质黄壤',
        '腐中层红泥质黄壤', '腐中层灰泥质黄壤', '腐厚层壤质黄色石灰土', '腐中层砾壤质钙质紫色土', '腐中层砾壤质酸性紫色土',
        '腐薄层红泥质黄壤', '腐中层砂泥质黄壤性土', '腐厚层壤质黑色石灰土'], dtype=object),
 (27012, 402))

In [15]:
sdt_data.shape,fzrt_data.shape,zrt_data.shape,dataset.shape,sdt_data.shape[0]+fzrt_data.shape[0]+zrt_data.shape[0]==dataset.shape[0]

((4058, 402), (1075, 402), (27012, 402), (32145, 402), True)

In [16]:
# 定义要检查的关键字列表
keywords = ['DEM','ndvi', 'PCA_0', 'Slope','Aspect', 'MRVBF','TopographicWetnessIndex', 'Mean',  'PH','PRE','SRA','TMP','VAP','WIND','PlanCurvature']

# 定义包含字符串的列表
strings = list(dataset.columns)

# 使用列表推导式和any函数检查并添加到新列表
filtered_strings = [s for s in strings if any(keyword in s for keyword in keywords)]
print(len(filtered_strings))
print(filtered_strings)


160
['DEM_COUNT', 'DEM_AREA', 'DEM_MIN', 'DEM_MAX', 'DEM_RANGE', 'DEM_MEAN', 'DEM_STD', 'DEM_SUM', 'DEM_MEDIAN', 'DEM_PCT90', 'Aspect_COUNT', 'Aspect_AREA', 'Aspect_MIN', 'Aspect_MAX', 'Aspect_RANGE', 'Aspect_MEAN', 'Aspect_STD', 'Aspect_SUM', 'Aspect_MEDIAN', 'Aspect_PCT90', 'PlanCurvature_COUNT', 'PlanCurvature_AREA', 'PlanCurvature_MIN', 'PlanCurvature_MAX', 'PlanCurvature_RANGE', 'PlanCurvature_MEAN', 'PlanCurvature_STD', 'PlanCurvature_SUM', 'PlanCurvature_MEDIAN', 'PlanCurvature_PCT90', 'RelativeSlopePosition_COUNT', 'RelativeSlopePosition_AREA', 'RelativeSlopePosition_MIN', 'RelativeSlopePosition_MAX', 'RelativeSlopePosition_RANGE', 'RelativeSlopePosition_MEAN', 'RelativeSlopePosition_STD', 'RelativeSlopePosition_SUM', 'RelativeSlopePosition_MEDIAN', 'RelativeSlopePosition_PCT90', 'Slope_COUNT', 'Slope_AREA', 'Slope_MIN', 'Slope_MAX', 'Slope_RANGE', 'Slope_MEAN', 'Slope_STD', 'Slope_SUM', 'Slope_MEDIAN', 'Slope_PCT90', 'TopographicWetnessIndex_COUNT', 'TopographicWetnessIndex_AR

In [17]:
# features = ['DEM', 'Mean', 'ndvi', 'PCA_0', 'LON', 'LAT', 'PH','DL','DZ']
# features = list(dataset.columns[1:-1])
sdt_features= ['DLMC','XMin','XMax','YMin','YMax','Centroid_X','Centroid_Y','Riverdis',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_AREA',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
fzrt_features = ['DLMC','XMin','XMax','YMin','YMax','Centroid_X','Centroid_Y',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_AREA',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
zrt_features = ['DLMC','XMin','XMax','YMin','YMax','Centroid_X','Centroid_Y',
           'DEM_MAX','DEM_MIN','DEM_MEAN','DEM_RANGE','DEM_AREA',
           'ndvi_MEAN',
           'PCA_0_MEAN',
           'Slope_MAX', 'Slope_MIN','Slope_MEAN','Slope_RANGE',
           'Aspect_MEAN',
           'MRVBF_MAX','MRVBF_MIN','MRVBF_MEAN','MRVBF_RANGE',
           'TopographicWetnessIndex_MEAN','TopographicWetnessIndex_RANGE',
           'Mean_MEAN',
           'PH_MEAN',
           'PRE_MEAN','PRE_RANGE',
           'SRA_MEAN','SRA_RANGE',
           'TMP_MEAN','TMP_RANGE',
           'VAP_MEAN','VAP_RANGE',
           'WIND_MEAN','WIND_RANGE',
           'PlanCurvature_MEAN','PlanCurvature_RANGE'
         ]
target = "TZ_label"

In [18]:
# 获取sdt数据集
sdt_data = sdt_data[sdt_features+[target]]

In [19]:
# 获取非自然土数据集
fzrt_data = fzrt_data[fzrt_features+[target]]

In [20]:
# 获取自然土数据集
zrt_data = zrt_data[zrt_features+[target]]

In [21]:
# 分割训练集和测试集
sdt_data_train,sdt_data_test = train_test_split(sdt_data, test_size=0.2, random_state=42)
fzrt_data_train,fzrt_data_test = train_test_split(fzrt_data, test_size=0.2, random_state=42)
zrt_data_train,zrt_data_test = train_test_split(zrt_data, test_size=0.2, random_state=42)

# data = data
label = "TZ_label"

In [22]:

hyperparameters={
	'NN_TORCH': {},
	'FASTAI': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}]
}


In [23]:
# 训练水稻土模型
sdt_model_path = os.path.join(model_path,'sdt_model')
# 检查路径是否存在，否则便创建
if not os.path.exists(sdt_model_path):
    os.makedirs(sdt_model_path)
else:
    print("文件夹已存在")
sdt_predictor = TabularPredictor(label=label,path=sdt_model_path,eval_metric="f1_weighted",problem_type='multiclass').fit(sdt_data_train,hyperparameters=hyperparameters)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\dy\modle\autogluon_type_class\sdt_model"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       50.39 GB / 63.81 GB (79.0%)
Disk Space Avail:   475

文件夹已存在


	0.6282	 = Validation score   (f1_weighted)
	3.61s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.8491	 = Validation score   (f1_weighted)
	0.72s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.8488	 = Validation score   (f1_weighted)
	0.78s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.8311	 = Validation score   (f1_weighted)
	0.58s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.8442	 = Validation score   (f1_weighted)
	0.61s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.6314	 = Validation score   (f1_weighted)
	16.4s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'RandomForestGini': 0.357, 'RandomForestEntr': 0.286, 'ExtraTreesGini': 0.214, 'ExtraTreesEntr': 0.143}
	0.8625	 = Validation score   (f1_weighted)
	0.11s	 

In [24]:
# 获取最佳模型
sdt_best_model = sdt_predictor.model_best
# 获取WeightedEnsemble_L3模型
# 获取模型的训练摘要
sdt_model_summary = sdt_predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.862541  f1_weighted       0.125925   2.798315                0.000000           0.114648            2       True          7
1     RandomForestGini   0.849114  f1_weighted       0.031251   0.716067                0.031251           0.716067            1       True          2
2     RandomForestEntr   0.848837  f1_weighted       0.031368   0.781605                0.031368           0.781605            1       True          3
3       ExtraTreesEntr   0.844162  f1_weighted       0.031395   0.607038                0.031395           0.607038            1       True          5
4       ExtraTreesGini   0.831065  f1_weighted       0.031910   0.578957                0.031910           0.578957            1       True          4
5       NeuralNetTorch   0.63143

In [25]:
sdt_predictor.leaderboard(sdt_data_train, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.970001,0.862541,f1_weighted,0.544159,0.125925,2.798315,0.015625,0.000000,0.114648,2,True,7
1,RandomForestGini,0.967857,0.849114,f1_weighted,0.193706,0.031251,0.716067,0.193706,0.031251,0.716067,1,True,2
2,RandomForestEntr,0.967846,0.848837,f1_weighted,0.101995,0.031368,0.781605,0.101995,0.031368,0.781605,1,True,3
3,ExtraTreesEntr,0.967520,0.844162,f1_weighted,0.114511,0.031395,0.607038,0.114511,0.031395,0.607038,1,True,5
4,ExtraTreesGini,0.965675,0.831065,f1_weighted,0.118322,0.031910,0.578957,0.118322,0.031910,0.578957,1,True,4
5,NeuralNetFastAI,0.819071,0.628237,f1_weighted,0.018893,0.016663,3.613287,0.018893,0.016663,3.613287,1,True,1
6,NeuralNetTorch,0.768980,0.631437,f1_weighted,0.016221,0.000000,16.402050,0.016221,0.000000,16.402050,1,True,6


In [26]:
sdt_predictor.leaderboard(sdt_data_test,silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.841286,0.862541,f1_weighted,0.385049,0.125925,2.798315,0.000000,0.000000,0.114648,2,True,7
1,RandomForestEntr,0.841058,0.848837,f1_weighted,0.082694,0.031368,0.781605,0.082694,0.031368,0.781605,1,True,3
2,RandomForestGini,0.833331,0.849114,f1_weighted,0.085197,0.031251,0.716067,0.085197,0.031251,0.716067,1,True,2
3,ExtraTreesEntr,0.825722,0.844162,f1_weighted,0.100989,0.031395,0.607038,0.100989,0.031395,0.607038,1,True,5
4,ExtraTreesGini,0.822682,0.831065,f1_weighted,0.116168,0.031910,0.578957,0.116168,0.031910,0.578957,1,True,4
5,NeuralNetFastAI,0.607632,0.628237,f1_weighted,0.011746,0.016663,3.613287,0.011746,0.016663,3.613287,1,True,1
6,NeuralNetTorch,0.584140,0.631437,f1_weighted,0.017133,0.000000,16.402050,0.017133,0.000000,16.402050,1,True,6


In [27]:
# 训练非自然土模型
fzrt_model_path = os.path.join(model_path,'fzrt_model')
# 检查路径是否存在，否则便创建
if not os.path.exists(fzrt_model_path):
    os.makedirs(fzrt_model_path)
else:
    print("文件夹已存在")
fzrt_predictor = TabularPredictor(label=label,path=fzrt_model_path,eval_metric="f1_weighted",problem_type='multiclass').fit(fzrt_data_train,hyperparameters=hyperparameters)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\dy\modle\autogluon_type_class\fzrt_model"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       50.22 GB / 63.81 GB (78.7%)
Disk Space Avail:   47

文件夹已存在


	0.578	 = Validation score   (f1_weighted)
	0.48s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7644	 = Validation score   (f1_weighted)
	0.48s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.8091	 = Validation score   (f1_weighted)
	0.51s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7917	 = Validation score   (f1_weighted)
	0.5s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.8068	 = Validation score   (f1_weighted)
	0.53s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.5789	 = Validation score   (f1_weighted)
	6.1s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'RandomForestEntr': 0.727, 'RandomForestGini': 0.182, 'NeuralNetFastAI': 0.091}
	0.8193	 = Validation score   (f1_weighted)
	0.1s	 = Training   runtime
	0.0s	 

In [28]:
# 获取最佳模型
fzrt_best_model = fzrt_predictor.model_best
# 获取WeightedEnsemble_L3模型
# 获取模型的训练摘要
fzrt_model_summary = fzrt_predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.819334  f1_weighted       0.073163  1.578421                0.000000           0.099077            2       True          7
1     RandomForestEntr   0.809070  f1_weighted       0.034233  0.513348                0.034233           0.513348            1       True          3
2       ExtraTreesEntr   0.806839  f1_weighted       0.034622  0.532060                0.034622           0.532060            1       True          5
3       ExtraTreesGini   0.791664  f1_weighted       0.034307  0.498351                0.034307           0.498351            1       True          4
4     RandomForestGini   0.764398  f1_weighted       0.033926  0.481244                0.033926           0.481244            1       True          2
5       NeuralNetTorch   0.578866  f1_

In [29]:
fzrt_predictor.leaderboard(fzrt_data_train, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.954069,0.819334,f1_weighted,0.132732,0.073163,1.578421,0.015004,0.000000,0.099077,2,True,7
1,RandomForestEntr,0.951818,0.809070,f1_weighted,0.047382,0.034233,0.513348,0.047382,0.034233,0.513348,1,True,3
2,ExtraTreesEntr,0.951647,0.806839,f1_weighted,0.051062,0.034622,0.532060,0.051062,0.034622,0.532060,1,True,5
3,ExtraTreesGini,0.948359,0.791664,f1_weighted,0.047892,0.034307,0.498351,0.047892,0.034307,0.498351,1,True,4
4,RandomForestGini,0.943618,0.764398,f1_weighted,0.060620,0.033926,0.481244,0.060620,0.033926,0.481244,1,True,2
5,NeuralNetTorch,0.832280,0.578866,f1_weighted,0.000664,0.003347,6.098684,0.000664,0.003347,6.098684,1,True,6
6,NeuralNetFastAI,0.802572,0.577971,f1_weighted,0.009726,0.005005,0.484752,0.009726,0.005005,0.484752,1,True,1


In [30]:
fzrt_predictor.leaderboard(fzrt_data_test,silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.798930,0.819334,f1_weighted,0.102096,0.073163,1.578421,0.001048,0.000000,0.099077,2,True,7
1,RandomForestEntr,0.792116,0.809070,f1_weighted,0.049607,0.034233,0.513348,0.049607,0.034233,0.513348,1,True,3
2,RandomForestGini,0.784807,0.764398,f1_weighted,0.051441,0.033926,0.481244,0.051441,0.033926,0.481244,1,True,2
3,ExtraTreesGini,0.763291,0.791664,f1_weighted,0.047184,0.034307,0.498351,0.047184,0.034307,0.498351,1,True,4
4,ExtraTreesEntr,0.749638,0.806839,f1_weighted,0.050897,0.034622,0.532060,0.050897,0.034622,0.532060,1,True,5
5,NeuralNetTorch,0.582765,0.578866,f1_weighted,0.011600,0.003347,6.098684,0.011600,0.003347,6.098684,1,True,6
6,NeuralNetFastAI,0.582407,0.577971,f1_weighted,0.000000,0.005005,0.484752,0.000000,0.005005,0.484752,1,True,1


In [31]:
# 训练自然土模型
zrt_model_path = os.path.join(model_path,'zrt_model')
# 检查路径是否存在，否则便创建
if not os.path.exists(zrt_model_path):
    os.makedirs(zrt_model_path)
else:
    print("文件夹已存在")
zrt_predictor = TabularPredictor(label=label,path=zrt_model_path,eval_metric="f1_weighted",problem_type='multiclass').fit(zrt_data_train,hyperparameters=hyperparameters)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\dy\modle\autogluon_type_class\zrt_model"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       50.20 GB / 63.81 GB (78.7%)
Disk Space Avail:   475

文件夹已存在


	0.8537	 = Validation score   (f1_weighted)
	12.68s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.9577	 = Validation score   (f1_weighted)
	2.25s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.9615	 = Validation score   (f1_weighted)
	4.46s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.9477	 = Validation score   (f1_weighted)
	0.89s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.952	 = Validation score   (f1_weighted)
	0.93s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.8749	 = Validation score   (f1_weighted)
	98.07s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'ExtraTreesEntr': 0.545, 'RandomForestEntr': 0.455}
	0.9628	 = Validation score   (f1_weighted)
	0.21s	 = Training   runtime
	0.0s	 = Validation runtime
A

In [32]:
# 获取最佳模型
zrt_best_model = zrt_predictor.model_best
# 获取WeightedEnsemble_L3模型
# 获取模型的训练摘要
zrt_model_summary = zrt_predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.962834  f1_weighted       0.098756   5.601427                0.000000           0.211102            2       True          7
1     RandomForestEntr   0.961523  f1_weighted       0.051885   4.463486                0.051885           4.463486            1       True          3
2     RandomForestGini   0.957727  f1_weighted       0.048505   2.250070                0.048505           2.250070            1       True          2
3       ExtraTreesEntr   0.952006  f1_weighted       0.046871   0.926838                0.046871           0.926838            1       True          5
4       ExtraTreesGini   0.947731  f1_weighted       0.047155   0.894323                0.047155           0.894323            1       True          4
5       NeuralNetTorch   0.87491

In [33]:
zrt_predictor.leaderboard(zrt_data_train, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.995742,0.962834,f1_weighted,0.785888,0.098756,5.601427,0.000000,0.000000,0.211102,2,True,7
1,RandomForestEntr,0.995604,0.961523,f1_weighted,0.282269,0.051885,4.463486,0.282269,0.051885,4.463486,1,True,3
2,RandomForestGini,0.995233,0.957727,f1_weighted,0.325366,0.048505,2.250070,0.325366,0.048505,2.250070,1,True,2
3,ExtraTreesEntr,0.994676,0.952006,f1_weighted,0.503619,0.046871,0.926838,0.503619,0.046871,0.926838,1,True,5
4,ExtraTreesGini,0.994259,0.947731,f1_weighted,0.492875,0.047155,0.894323,0.492875,0.047155,0.894323,1,True,4
5,NeuralNetTorch,0.910939,0.874917,f1_weighted,0.110664,0.017991,98.066746,0.110664,0.017991,98.066746,1,True,6
6,NeuralNetFastAI,0.894918,0.853720,f1_weighted,0.161989,0.016539,12.683930,0.161989,0.016539,12.683930,1,True,1


In [34]:
zrt_predictor.leaderboard(zrt_data_test,silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr,0.955300,0.961523,f1_weighted,0.194372,0.051885,4.463486,0.194372,0.051885,4.463486,1,True,3
1,RandomForestGini,0.954304,0.957727,f1_weighted,0.234123,0.048505,2.250070,0.234123,0.048505,2.250070,1,True,2
2,WeightedEnsemble_L2,0.951816,0.962834,f1_weighted,0.524045,0.098756,5.601427,0.000000,0.000000,0.211102,2,True,7
3,ExtraTreesEntr,0.944618,0.952006,f1_weighted,0.329674,0.046871,0.926838,0.329674,0.046871,0.926838,1,True,5
4,ExtraTreesGini,0.941234,0.947731,f1_weighted,0.333332,0.047155,0.894323,0.333332,0.047155,0.894323,1,True,4
5,NeuralNetTorch,0.864418,0.874917,f1_weighted,0.035279,0.017991,98.066746,0.035279,0.017991,98.066746,1,True,6
6,NeuralNetFastAI,0.849372,0.853720,f1_weighted,0.040371,0.016539,12.683930,0.040371,0.016539,12.683930,1,True,1


In [35]:
# 获取sdt模型特征重要性
sdt_feature_importance = sdt_predictor.feature_importance(data=sdt_data_train)
pd.DataFrame(sdt_feature_importance)

Computing feature importance via permutation shuffling for 40 features using 3225 rows with 5 shuffle sets...
	75.01s	= Expected runtime (15.0s per shuffle set)
	20.61s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
XMax,0.009935,0.000637,0.000002,5,0.011246,0.008623
XMin,0.009171,0.000854,0.000009,5,0.010929,0.007413
Centroid_X,0.009124,0.001284,0.000046,5,0.011769,0.006480
YMax,0.006329,0.001850,0.000784,5,0.010137,0.002521
YMin,0.006201,0.000868,0.000045,5,0.007988,0.004414
Centroid_Y,0.006011,0.001716,0.000717,5,0.009545,0.002478
DEM_MIN,0.005659,0.001128,0.000180,5,0.007981,0.003337
SRA_MEAN,0.005567,0.000885,0.000074,5,0.007390,0.003744
DEM_MEAN,0.005475,0.000476,0.000007,5,0.006454,0.004495
PRE_MEAN,0.004620,0.000408,0.000007,5,0.005459,0.003780


In [36]:
# 获取fzrt模型特征重要性
fzrt_feature_importance = fzrt_predictor.feature_importance(data=fzrt_data_train)
pd.DataFrame(fzrt_feature_importance)

Computing feature importance via permutation shuffling for 39 features using 853 rows with 5 shuffle sets...
	25.72s	= Expected runtime (5.14s per shuffle set)
	3.18s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
TMP_MEAN,1.104366e-02,0.003197,0.000756,5,0.017626,0.004462
Centroid_Y,1.001878e-02,0.001391,0.000043,5,0.012883,0.007154
XMax,9.003374e-03,0.003820,0.003105,5,0.016868,0.001138
YMin,7.854019e-03,0.001343,0.000099,5,0.010619,0.005089
SRA_MEAN,7.530263e-03,0.003472,0.004172,5,0.014680,0.000381
YMax,7.390230e-03,0.000993,0.000038,5,0.009436,0.005345
XMin,6.884965e-03,0.001285,0.000139,5,0.009531,0.004239
Centroid_X,6.600991e-03,0.002710,0.002761,5,0.012182,0.001020
VAP_MEAN,6.143723e-03,0.001941,0.001052,5,0.010140,0.002147
TopographicWetnessIndex_RANGE,5.430089e-03,0.001342,0.000413,5,0.008193,0.002667


In [37]:
# 获取zrt模型特征重要性
zrt_feature_importance = zrt_predictor.feature_importance(data=zrt_data_train)
pd.DataFrame(zrt_feature_importance)

Computing feature importance via permutation shuffling for 39 features using 5000 rows with 5 shuffle sets...
	118.74s	= Expected runtime (23.75s per shuffle set)
	25.63s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
XMax,1.299584e-02,0.002255,0.000105,5,0.017640,0.008352
XMin,9.094121e-03,0.002423,0.000551,5,0.014083,0.004106
Centroid_X,8.414316e-03,0.001077,0.000031,5,0.010631,0.006197
YMin,7.198272e-03,0.001037,0.000050,5,0.009334,0.005063
Centroid_Y,6.883316e-03,0.000938,0.000040,5,0.008815,0.004952
YMax,6.636600e-03,0.001021,0.000065,5,0.008740,0.004534
SRA_MEAN,3.732687e-03,0.000812,0.000252,5,0.005404,0.002061
PRE_MEAN,2.011352e-03,0.000558,0.000642,5,0.003160,0.000863
WIND_MEAN,1.607989e-03,0.000636,0.002417,5,0.002918,0.000298
TMP_MEAN,1.568812e-03,0.000499,0.001078,5,0.002596,0.000542
